In [ ]:
import math
import numpy
import numpy as np
from typing import Union
import collections

from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier 
from sklearn.datasets import load_iris  


class DecisionNode(object):
    def __init__(self, f_idx, threshold, value=None, L=None, R=None):
        self.f_idx = f_idx  
        self.threshold = threshold  
        self.value = value 
        self.L = L  
        self.R = R  


def find_best_threshold(dataset: np.ndarray, f_idx: int, split_choice: str):  
    best_gain = -math.inf 
    best_gini = math.inf
    best_threshold = None
    dataset_sorted = sorted(list(set(dataset[:, f_idx].reshape(-1)))) 
    candidate = [] 

    for i in range(len(dataset_sorted) - 1):
        candidate.append(round((dataset_sorted[i] + dataset_sorted[i + 1]) / 2.0, 2))

    for threshold in candidate:
        L, R = split_dataset(dataset, f_idx, threshold)  
        gain = None
        if split_choice == "gain":
            gain = calculate_gain(dataset, L, R)  
            if gain > best_gain:  
                best_gain = gain
                best_threshold = threshold
        if split_choice == "gain_ratio":
            gain = calculate_gain_ratio(dataset, L, R)
            if gain > best_gain:  
                best_gain = gain
                best_threshold = threshold
        if split_choice == "gini":
            gini = calculate_gini_index(dataset, L, R)
            if gini < best_gini:  
                best_gini = gini
                best_threshold = threshold

    return best_threshold, best_gain


def calculate_entropy(dataset: np.ndarray): 
    scale = dataset.shape[0]  
    d = {}
    for data in dataset:
        key = data[-1]
        if key in d:
            d[key] += 1
        else:
            d[key] = 1

    entropy = 0.0
    for key in d.keys():
        p = d[key] / scale
        entropy -= p * math.log(p, 2)
    return entropy


def calculate_gain(dataset, l, r):
    e1 = calculate_entropy(dataset)
    e2 = len(l) / len(dataset) * calculate_entropy(l) + len(r) / len(dataset) * calculate_entropy(r)
    gain = e1 - e2
    return gain


def calculate_gain_ratio(dataset, l, r):
    gain = calculate_gain(dataset, l, r)
    p1 = len(l) / len(dataset)
    p2 = len(r) / len(dataset)
   
    if p1 == 0:
        s = p2 * math.log(p2, 2)
    elif p2 == 0:
        s = p1 * math.log(p1, 2)
    else:
        s = - p1 * math.log(p1, 2) - p2 * math.log(p2, 2)

    gain_ratio = gain / s
    return gain_ratio


def calculate_gini(dataset: np.ndarray):
    scale = dataset.shape[0] 
    d = {}
    for data in dataset:
        key = data[-1]
        if key in d:
            d[key] += 1
        else:
            d[key] = 1

    gini = 1.0
    for key in d.keys():
        p = d[key] / scale
        gini -= p * p
    return gini


def calculate_gini_index(dataset, l, r):
    gini_index = len(l) / len(dataset) * calculate_gini(l) + len(r) / len(dataset) * calculate_gini(r)
    return gini_index


def split_dataset(X: np.ndarray, f_idx: int, threshold: float):
    L = X[:, f_idx] < threshold
    R = ~L
    return X[L], X[R]


def majority_count(dataset):
    class_list = [data[-1] for data in dataset]
    return collections.Counter(class_list).most_common(1)[0][0]


def build_tree(dataset: np.ndarray, f_idx_list: list, split_choice: str):

    class_list = [data[-1] for data in dataset]
    if class_list.count(class_list[0]) == len(class_list):
        return DecisionNode(None, None, value=class_list[0])
    elif len(f_idx_list) == 0:
        value = collections.Counter(class_list).most_common(1)[0][0]
        return DecisionNode(None, None, value=value)

    else:
        best_gain = -math.inf
        best_gini = math.inf
        best_threshold = None
        best_f_idx = None

        for i in f_idx_list:
            threshold, gain = find_best_threshold(dataset, i, split_choice)
            if split_choice == "gini":
                if gain < best_gini:
                    best_gini = gain
                    best_threshold = threshold
                    best_f_idx = i
            if split_choice == "gain" or split_choice == "gain_ratio" :
                if gain > best_gain:  
                    best_gain = gain
                    best_threshold = threshold
                    best_f_idx = i

        son_f_idx_list = f_idx_list.copy()
        son_f_idx_list.remove(best_f_idx)

        L, R = split_dataset(dataset, best_f_idx, best_threshold)
        if len(L) == 0:
            L_tree = DecisionNode(None, None, majority_count(dataset))  
        else:
            L_tree = build_tree(L, son_f_idx_list, split_choice) 

        if len(R) == 0:
            R_tree = DecisionNode(None, None, majority_count(dataset)) 
        else:
            R_tree = build_tree(R, son_f_idx_list, split_choice)  
        return DecisionNode(best_f_idx, best_threshold, value=None, L=L_tree, R=R_tree)


def predict_one(model: DecisionNode, data):
    if model.value is not None:
        return model.value
    else:
        feature_one = data[model.f_idx]
        branch = None
        if feature_one >= model.threshold:
            branch = model.R 
        else:
            branch = model.L  
        return predict_one(branch, data)


def predict_accuracy(y_predict, y_test):
    y_predict = y_predict.tolist()
    y_test = y_test.tolist()
    count = 0
    for i in range(len(y_predict)):
        if int(y_predict[i]) == y_test[i]:
            count = count + 1
    accuracy = count / len(y_predict)
    return accuracy


class SimpleDecisionTree(object):
    def __init__(self, split_choice, min_samples: int = 1, min_gain: float = 0, max_depth: Union[int, None] = None,
                 max_leaves: Union[int, None] = None):
        self.split_choice = split_choice

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        dataset_in = np.c_[X, y]
        f_idx_list = [i for i in range(X.shape[1])]
        self.my_tree = build_tree(dataset_in, f_idx_list, self.split_choice)

    def predict(self, X: np.ndarray) -> np.ndarray:
        predict_list = []
        for data in X:
            predict_list.append(predict_one(self.my_tree, data))

        return np.array(predict_list)


if __name__ == "__main__":

    predict_accuracy_all = []

    for i in range(10):
        iris = load_iris() 
        x = iris.data  
        y = iris.target 
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

        predict_accuracy_list = []  
        split_choice_list = ["gain", "gain_ratio", "gini"]
        for split_choice in split_choice_list:
            m = SimpleDecisionTree(split_choice)
            m.fit(X_train, y_train)
            y_predict = m.predict(X_test)
            y_predict_accuracy = predict_accuracy(y_predict, y_test.reshape(-1))
            predict_accuracy_list.append(y_predict_accuracy)

           
        clf = DecisionTreeClassifier()  
        clf.fit(X_train, y_train) 
        predicted = clf.predict(X_test)
        predict_accuracy_list.append(clf.score(X_test, y_test))
        predict_accuracy_all.append(predict_accuracy_list)

    p = numpy.array(predict_accuracy_all)
    p = np.round(p, decimals=3)
    for i in p:
        print(i)
    print(p.mean(axis=0))
